In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')

# ↓ 若已解壓可跳過
!unzip -q "/content/drive/MyDrive/Colab Notebooks/8_4_8dataset/cor.zip" -d /content

Mounted at /content/drive


In [ ]:
!unzip -q "/content/drive/MyDrive/Colab Notebooks/新csv.zip" -d /content

In [ ]:
import os, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold

# 隨機種子（可固定）
seed = np.random.randint(1, 10000)
print(f" 本次使用隨機種子：{seed}")

# 原始資料
full_df = pd.read_csv("/content/新csv/8_4_9dataset.csv")
print(f" 原始資料筆數：{len(full_df)}")

# STEP 2. 切出 10% test
train_val_df, test_df = train_test_split(full_df, test_size=0.15, random_state=seed, shuffle=True)
test_df.to_csv("/content/8_4_8dataset_12ch/csv/test.csv", index=False)
print(f"測試集切分完成：Test = {len(test_df)}")

# STEP 3. 對剩下資料做 5-fold split
kf = KFold(n_splits=5, shuffle=True, random_state=seed + 42)
for fold, (train_idx, val_idx) in enumerate(kf.split(train_val_df)):
    train_df = train_val_df.iloc[train_idx].reset_index(drop=True)
    val_df   = train_val_df.iloc[val_idx].reset_index(drop=True)
    train_df.to_csv(f"/content/8_4_8dataset_12ch/csv/fold{fold+1}_train.csv", index=False)
    val_df.to_csv(f"/content/8_4_8dataset_12ch/csv/fold{fold+1}_val.csv", index=False)
    print(f"Fold {fold+1} 完成：Train = {len(train_df)}, Val = {len(val_df)}")


 本次使用隨機種子：2908
 原始資料筆數：7649
測試集切分完成：Test = 1148
Fold 1 完成：Train = 5200, Val = 1301
Fold 2 完成：Train = 5201, Val = 1300
Fold 3 完成：Train = 5201, Val = 1300
Fold 4 完成：Train = 5201, Val = 1300
Fold 5 完成：Train = 5201, Val = 1300


In [ ]:
import glob
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm
from pathlib import Path
import matplotlib.pyplot as plt
import os

class Config:
    BATCH_SIZE = 4
    IMG_DIR = "/content/cor"
    TEST_CSV = "/content/8_4_8dataset_12ch/csv/test.csv"
    SAVE_DIR = "/content/drive/MyDrive/brain_age_models"
    N_SPLITS = 5

os.makedirs(Config.SAVE_DIR, exist_ok=True)

cfg = Config()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
################################################################################
# 在現有代碼中集成加權損失 - 最小改動版本
################################################################################

# 1. 添加年齡組計算函數
def get_age_group(age):
    """將年齡映射到年齡組"""
    if age < 5: return "0-4"
    elif age < 10: return "5-9"
    elif age < 15: return "10-14"
    elif age < 20: return "15-19"
    elif age < 25: return "20-24"
    elif age < 30: return "25-29"
    elif age < 35: return "30-34"
    elif age < 40: return "35-39"
    elif age < 45: return "40-44"
    elif age < 50: return "45-49"
    elif age < 55: return "50-54"
    elif age < 60: return "55-59"
    elif age < 65: return "60-64"
    elif age < 70: return "65-69"
    elif age < 75: return "70-74"
    elif age < 80: return "75-79"
    elif age < 85: return "80-84"
    elif age < 90: return "85-89"
    elif age < 95: return "90-94"
    else: return "95-99"

# 2. 計算年齡組權重
def calculate_age_weights(df, smoothing=0.5, max_weight=20.0):
    """
    計算年齡組權重
    Args:
        df: 包含AGE列的DataFrame
        smoothing: 平滑參數 (0.1-1.0)，越小權重差異越大，越大權重越平均
        max_weight: 權重上限，避免極端樣本過度主導
    """
    df['age_group'] = df['AGE'].apply(get_age_group)
    age_group_counts = df['age_group'].value_counts().to_dict()
    total_samples = len(df)

    # 計算權重：權重與樣本數成反比，並加入平滑
    age_group_weights = {}
    for group, count in age_group_counts.items():
        raw_weight = total_samples / count
        # 平滑處理：權重 = (1-smoothing) * 原始權重 + smoothing * 1.0
        smoothed_weight = (1 - smoothing) * raw_weight + smoothing
        #  新增：設置權重上限
        age_group_weights[group] = min(smoothed_weight, max_weight)

    #  修正：直接返回年齡組權重字典，不是個別年齡權重
    print(" 年齡組權重分佈:")
    for group in sorted(age_group_weights.keys(), key=lambda x: int(x.split('-')[0])):
        count = age_group_counts[group]
        weight = age_group_weights[group]
        raw_weight = total_samples / count
        smoothed_weight = (1 - smoothing) * raw_weight + smoothing

        # 顯示是否被限制
        capped_indicator = " (限制)" if smoothed_weight > max_weight else ""
        print(f"  {group:>6}: {count:>4} 樣本, 權重 = {weight:.3f}{capped_indicator}")

    return age_group_weights

In [ ]:
class GlobalOnlyDatasetWeighted(Dataset):
    def __init__(self, df: pd.DataFrame, img_dir: str, age_group_weights_dict=None):
        self.df = df.reset_index(drop=True)
        self.dir = Path(img_dir)
        self.age_group_weights_dict = age_group_weights_dict or {}

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        arr = np.load(self.dir / f"{row['NIfTI_ID']}.npy")
        if arr.shape[0] != 6:
            arr = arr.transpose(2, 0, 1)

        img = torch.tensor(arr, dtype=torch.float32)
        age = torch.tensor(row['AGE'], dtype=torch.float32)
        sex = torch.tensor(row['SEX'], dtype=torch.float32)

        #  用年齡組獲取權重
        age_group = get_age_group(row['AGE'])
        weight = self.age_group_weights_dict.get(age_group, 1.0)
        sample_weight = torch.tensor(weight, dtype=torch.float32)

        return img, age, sex, sample_weight, row['NIfTI_ID']

# ===============================================
# STEP 2. Dataset 定義
# ===============================================
class GlobalOnlyDataset(Dataset):
    def __init__(self, df: pd.DataFrame, img_dir: str):
        self.df = df.reset_index(drop=True)
        self.dir = Path(img_dir)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        arr = np.load(self.dir / f"{row['NIfTI_ID']}.npy")
        if arr.shape[0] != 6:
            arr = arr.transpose(2, 0, 1)
        img = torch.tensor(arr, dtype=torch.float32)
        age = torch.tensor(row['AGE'], dtype=torch.float32)
        sex = torch.tensor(row['SEX'], dtype=torch.float32)
        return img, age, sex, row['NIfTI_ID']

class MixupDualDataset(Dataset):
    def __init__(self, dataset, alpha=0.4):
        self.original = dataset
        self.alpha = alpha
        self.df = dataset.df.reset_index(drop=True)
        self.dir = dataset.dir
        self.sex0_indices = self.df[self.df['SEX'] == 0].index.tolist()
        self.sex1_indices = self.df[self.df['SEX'] == 1].index.tolist()

    def __len__(self):
        return len(self.original) * 2

    def __getitem__(self, idx):
        if idx < len(self.original):
            return self.original[idx]
        else:
            mix_idx = idx - len(self.original)
            anchor = self.df.iloc[mix_idx]
            anchor_sex = anchor['SEX']
            pool = self.sex0_indices if anchor_sex == 0 else self.sex1_indices
            pair_idx = np.random.choice(pool)
            pair = self.df.iloc[pair_idx]
            lam = np.random.beta(self.alpha, self.alpha)
            arr1 = np.load(self.dir / f"{anchor['NIfTI_ID']}.npy")
            arr2 = np.load(self.dir / f"{pair['NIfTI_ID']}.npy")
            if arr1.shape[0] != 16: arr1 = arr1.transpose(2, 0, 1)
            if arr2.shape[0] != 16: arr2 = arr2.transpose(2, 0, 1)
            mixed_img = lam * arr1 + (1 - lam) * arr2
            mixed_age = lam * anchor['AGE'] + (1 - lam) * pair['AGE']
            img = torch.tensor(mixed_img, dtype=torch.float32)
            age = torch.tensor(mixed_age, dtype=torch.float32)
            sex = torch.tensor(anchor_sex, dtype=torch.float32)
            label = f"Mix({anchor['NIfTI_ID']},{pair['NIfTI_ID']},λ={lam:.2f})"
            return img, age, sex, label



In [ ]:
class BrainAgeModelWithSex(nn.Module):
    def __init__(self, in_ch=6, hid=128):
        super().__init__()
        self.block1 = nn.Sequential(nn.Conv2d(in_ch, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2))
        self.block2 = nn.Sequential(nn.Conv2d(64, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(), nn.MaxPool2d(2))
        self.block3 = nn.Sequential(nn.Conv2d(128, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU())
        self.gap = nn.AdaptiveAvgPool2d(1)
        with torch.no_grad():
            dummy = torch.zeros(1, in_ch, 218, 218)
            x = self.block1(dummy)
            x = self.block2(x)
            x = self.block3(x)
            x = self.gap(x).flatten(1)
            img_feature_dim = x.shape[1]
        self.fc = nn.Sequential(
            nn.Linear(img_feature_dim + 1, hid), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(hid, hid // 2), nn.ReLU(),
            nn.Linear(hid // 2, 1)
        )

    def forward(self, x, sex):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.gap(x).flatten(1)
        x = torch.cat([x, sex.unsqueeze(1)], dim=1)
        return self.fc(x).squeeze(-1)



In [ ]:
def mse_loss(pred, target):
    return 0.5 * torch.mean((pred - target) ** 2)

def mae_loss(pred, target):
    return torch.mean(torch.abs(pred - target))

from tqdm import tqdm

def train_one_epoch_weighted(model, loader, opt, loss_type='mae', use_weights=True, scheduler=None):
    model.train()
    total_loss, total_raw_mae, total_samples = 0.0, 0.0, 0

    for batch in tqdm(loader, desc=f'Train({loss_type.upper()})', leave=False):
        if len(batch) == 5:
            imgs, ages, sex, weights, _ = batch
            weights = weights.to(device)
        else:
            imgs, ages, sex, _ = batch
            weights = torch.ones_like(ages)

        imgs, ages, sex = imgs.to(device), ages.to(device), sex.to(device)

        if use_weights:
            weights = weights / (weights.mean() + 1e-8)

        opt.zero_grad(set_to_none=True)
        preds = model(imgs, sex)

        if loss_type == 'mse':
            errors = 0.5 * (preds - ages) ** 2
        else:
            errors = torch.abs(preds - ages)

        loss = (weights * errors).mean()
        raw_mae = torch.mean(torch.abs(preds - ages)).item()

        loss.backward()
        opt.step()

        if scheduler is not None:
            scheduler.step()

        total_loss += loss.item() * len(ages)
        total_raw_mae += raw_mae * len(ages)
        total_samples += len(ages)

    avg_loss = total_loss / total_samples
    avg_raw_mae = total_raw_mae / total_samples
    return avg_loss, avg_raw_mae

def validate_one_epoch(model, loader):
    model.eval()
    total_mae, total_mse, n = 0.0, 0.0, 0

    with torch.no_grad():
        for batch_data in tqdm(loader, desc='Val', leave=False):
            if len(batch_data) == 5:
                imgs, ages, sex, _, _ = batch_data
            else:
                imgs, ages, sex, _ = batch_data

            imgs, ages, sex = imgs.to(device), ages.to(device), sex.to(device)
            pred = model(imgs, sex)

            mae = torch.mean(torch.abs(pred - ages))
            mse = torch.mean((pred - ages) ** 2)

            total_mae += mae.item() * len(ages)
            total_mse += mse.item() * len(ages)
            n += len(ages)

    return total_mae / n, total_mse / n

In [ ]:
def train_with_weights(loss_type: str = 'mae'):

    TOTAL_EPOCHS = 400
    MAX_LR = 5e-4
    PCT_START = 0.1
    DIV_FACTOR = 5
    FINAL_DIV_FACTOR = 20

    for fold in range(1, 6):
        print(f"\n開始訓練 Fold {fold} (加權 {loss_type.upper()} 損失)...")

        train_df = pd.read_csv(f"/content/8_4_8dataset_12ch/csv/fold{fold}_train.csv")
        val_df = pd.read_csv(f"/content/8_4_8dataset_12ch/csv/fold{fold}_val.csv")

        age_group_weights = calculate_age_weights(train_df, smoothing=1, max_weight=80.0)

        train_ds = GlobalOnlyDatasetWeighted(train_df, cfg.IMG_DIR, age_group_weights)
        val_ds = GlobalOnlyDataset(val_df, cfg.IMG_DIR)

        train_ld = DataLoader(train_ds, batch_size=cfg.BATCH_SIZE, shuffle=True, num_workers=2)
        val_ld = DataLoader(val_ds, batch_size=cfg.BATCH_SIZE, shuffle=False, num_workers=2)

        model = BrainAgeModelWithSex().to(device)

        base_lr = MAX_LR / DIV_FACTOR
        opt = optim.Adam(model.parameters(), lr=base_lr, weight_decay=1e-5)

        steps_per_epoch = len(train_ld)
        scheduler = optim.lr_scheduler.OneCycleLR(
            opt,
            max_lr=MAX_LR,
            epochs=TOTAL_EPOCHS,
            steps_per_epoch=steps_per_epoch,
            pct_start=PCT_START,
            div_factor=DIV_FACTOR,
            final_div_factor=FINAL_DIV_FACTOR,
            anneal_strategy='cos',
            three_phase=False
        )

        best_val_mae = float('inf')
        best_state = None

        train_hist, val_mae_hist, val_mse_hist = [], [], []

        try:
            for epoch in range(1, TOTAL_EPOCHS + 1):
                curr_lr = opt.param_groups[0]['lr']
                print(f"\n[FOLD {fold}] === Epoch {epoch}/{TOTAL_EPOCHS} ===  (LR: {curr_lr:.6g})")

                train_loss, train_raw_mae = train_one_epoch_weighted(
                    model, train_ld, opt,
                    loss_type=loss_type,
                    use_weights=True,
                    scheduler=scheduler
                )

                val_mae, val_mse = validate_one_epoch(model, val_ld)

                train_hist.append(train_loss)
                val_mae_hist.append(val_mae)
                val_mse_hist.append(val_mse)

                print(f"Train Loss(加權): {train_loss:.4f} | "
                      f"Train Raw MAE: {train_raw_mae:.4f} | "
                      f"Val MAE: {val_mae:.4f}")

                if val_mae < best_val_mae:
                    best_val_mae = val_mae
                    best_state = model.state_dict()
                    print(f"New best Val MAE: {val_mae:.4f}")

        except KeyboardInterrupt:
            print(f"手動中斷 Fold {fold}")

        finally:
            if best_state is not None:
                torch.save(best_state, f"{cfg.SAVE_DIR}/fold{fold}_{loss_type}_best.pth")
                print(f"Fold {fold} 最佳模型已儲存")

            if train_hist:
                plt.figure(figsize=(12, 5))

                plt.subplot(1, 2, 1)
                plt.plot(train_hist, label=f'Train {loss_type.upper()}', color='blue')
                plt.plot(val_mae_hist, label='Val MAE', color='red')
                plt.xlabel('Epoch'); plt.ylabel('Loss / Error'); plt.grid(alpha=0.3); plt.legend()

                plt.subplot(1, 2, 2)
                plt.plot(val_mae_hist, label='Val MAE', color='red')
                plt.plot([np.sqrt(m) for m in val_mse_hist], label='Val RMSE', color='orange')
                plt.xlabel('Epoch'); plt.ylabel('Error'); plt.grid(alpha=0.3); plt.legend()

                plt.tight_layout()
                plt.savefig(f"{cfg.SAVE_DIR}/fold{fold}_{loss_type}_curves.png", dpi=150)
                plt.close()

            if 'model' in locals():
                model.cpu()
            for vname in ['model', 'opt', 'scheduler', 'train_ld', 'val_ld', 'train_ds', 'val_ds']:
                if vname in locals():
                    del locals()[vname]
                elif vname in globals():
                    del globals()[vname]
            torch.cuda.empty_cache()

In [ ]:
def analyze_age_group_performance(model_path, test_csv, img_dir):
    test_df = pd.read_csv(test_csv)
    test_ds = GlobalOnlyDataset(test_df, img_dir)
    test_ld = DataLoader(test_ds, batch_size=4, shuffle=False, num_workers=2)

    model = BrainAgeModelWithSex().to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    preds, gts = [], []
    with torch.no_grad():
        for imgs, ages, sex, _ in tqdm(test_ld, desc='Testing'):
            output = model(imgs.to(device), sex.to(device)).cpu().numpy()
            preds.extend(output.tolist())
            gts.extend(ages.numpy().tolist())

    test_df['predicted_age'] = preds
    test_df['age_group'] = test_df['AGE'].apply(get_age_group)
    test_df['error'] = test_df['predicted_age'] - test_df['AGE']
    test_df['abs_error'] = np.abs(test_df['error'])
    test_df['squared_error'] = test_df['error'] ** 2

    print(f"\n年齡組詳細表現分析 ({os.path.basename(model_path)}):")
    print(f"{'Age Group':>10} {'Count':>6} {'MAE':>8} {'RMSE':>8} {'Bias':>8} {'Low Est %':>10}")
    print("-" * 62)

    results = []
    for age_group in sorted(test_df['age_group'].unique(), key=lambda x: int(x.split('-')[0])):
        group_data = test_df[test_df['age_group'] == age_group]
        if len(group_data) > 0:
            mae = group_data['abs_error'].mean()
            rmse = np.sqrt(group_data['squared_error'].mean())
            bias = group_data['error'].mean()
            count = len(group_data)

            underestimate_pct = (group_data['error'] < 0).mean() * 100

            results.append({
                'age_group': age_group,
                'count': count,
                'mae': mae,
                'rmse': rmse,
                'bias': bias,
                'underestimate_pct': underestimate_pct
            })

            print(f"{age_group:>10} {count:>6} {mae:>8.3f} {rmse:>8.3f} {bias:>8.3f} {underestimate_pct:>9.1f}%")

    overall_mae = test_df['abs_error'].mean()
    overall_rmse = np.sqrt(test_df['squared_error'].mean())
    overall_bias = test_df['error'].mean()
    overall_underestimate = (test_df['error'] < 0).mean() * 100

    print("-" * 62)
    print(f"{'Overall':>10} {len(test_df):>6} {overall_mae:>8.3f} {overall_rmse:>8.3f} {overall_bias:>8.3f} {overall_underestimate:>9.1f}%")

    elderly_data = test_df[test_df['AGE'] >= 65]
    if len(elderly_data) > 0:
        elderly_mae = elderly_data['abs_error'].mean()
        elderly_bias = elderly_data['error'].mean()
        elderly_under = (elderly_data['error'] < 0).mean() * 100
        print(f"\n老年組 (>=65歲) 特別分析:")
        print(f"   樣本數: {len(elderly_data)}")
        print(f"   MAE: {elderly_mae:.3f}")
        print(f"   平均偏差: {elderly_bias:.3f} ({'低估' if elderly_bias < 0 else '高估'})")
        print(f"   低估比例: {elderly_under:.1f}%")

    middle_data = test_df[(test_df['AGE'] >= 35) & (test_df['AGE'] <= 59)]
    if len(middle_data) > 0:
        middle_mae = middle_data['abs_error'].mean()
        middle_bias = middle_data['error'].mean()
        middle_under = (middle_data['error'] < 0).mean() * 100
        print(f"\n中年組 (40–64歲) 特別分析:")
        print(f"   樣本數: {len(middle_data)}")
        print(f"   MAE: {middle_mae:.3f}")
        print(f"   平均偏差: {middle_bias:.3f} ({'低估' if middle_bias < 0 else '高估'})")
        print(f"   低估比例: {middle_under:.1f}%")

    return results

In [ ]:
if __name__ == "__main__":
    loss_functions = ['mae']
    for loss_type in loss_functions:
        print(f"\n" + "="*60)
        print(f"開始 {loss_type.upper()} 訓練...")
        print("="*60)

        train_with_weights(loss_type)

        print(f"\n分析 {loss_type.upper()} 訓練結果...")
        best_models = glob.glob(f"{cfg.SAVE_DIR}/fold*_{loss_type}_best.pth")
        for model_path in best_models:
            analyze_age_group_performance(model_path, cfg.TEST_CSV, cfg.IMG_DIR)

    print("\nMAE版本的加權訓練完成！")


📂 所有模型將統一儲存在: /content/drive/MyDrive/Colab Notebooks/8_4_model_pth/cor_6ch_smooting_1

🚀 開始 MAE 訓練 (統一存檔)...

🚀 開始訓練 Fold 1 (論文式加權 MAE 損失)...
📊 年齡組權重分佈:
     5-9:  624 樣本, 權重 = 1.000
   10-14:  645 樣本, 權重 = 1.000
   15-19:  877 樣本, 權重 = 1.000
   20-24: 1371 樣本, 權重 = 1.000
   25-29:  434 樣本, 權重 = 1.000
   30-34:  148 樣本, 權重 = 1.000
   35-39:   53 樣本, 權重 = 1.000
   40-44:   48 樣本, 權重 = 1.000
   45-49:   70 樣本, 權重 = 1.000
   50-54:   65 樣本, 權重 = 1.000
   55-59:  102 樣本, 權重 = 1.000
   60-64:  134 樣本, 權重 = 1.000
   65-69:  219 樣本, 權重 = 1.000
   70-74:  218 樣本, 權重 = 1.000
   75-79:  115 樣本, 權重 = 1.000
   80-84:   52 樣本, 權重 = 1.000
   85-89:   26 樣本, 權重 = 1.000
   90-94:    2 樣本, 權重 = 1.000
   95-99:    1 樣本, 權重 = 1.000

[FOLD 1] === Epoch 1/400 ===  (LR: 0.0001)


Train Loss(加權): 14.0332 | Train Raw MAE: 14.0332 | Val MAE: 8.9715
🎯 New best Val MAE: 8.9715

[FOLD 1] === Epoch 2/400 ===  (LR: 0.000100617)


Train Loss(加權): 10.9616 | Train Raw MAE: 10.9616 | Val MAE: 8.4836
🎯 New best Val MAE: 8.4836

[FOLD 1] === Epoch 3/400 ===  (LR: 0.000102462)


Train Loss(加權): 10.6251 | Train Raw MAE: 10.6251 | Val MAE: 7.9599
🎯 New best Val MAE: 7.9599

[FOLD 1] === Epoch 4/400 ===  (LR: 0.000105526)


Train Loss(加權): 10.3635 | Train Raw MAE: 10.3635 | Val MAE: 7.9219
🎯 New best Val MAE: 7.9219

[FOLD 1] === Epoch 5/400 ===  (LR: 0.000109789)


...


📊 年齡組詳細表現分析 (fold3_mae_best.pth):
 Age Group  Count      MAE     RMSE     Bias  Low Est %
--------------------------------------------------------------
       5-9    152    1.315    1.886    0.662      40.1%
     10-14    133    1.568    1.978   -0.544      60.2%
     15-19    192    1.941    2.651    0.892      22.9%
     20-24    291    1.842    3.338   -0.206      67.0%
     25-29     79    3.939    4.804   -1.891      79.7%
     30-34     37    6.435    7.913   -2.853      70.3%
     35-39     14    7.548    8.801   -1.445      57.1%
     40-44     10    8.501   10.226   -4.810      60.0%
     45-49     13   10.361   11.792    5.846      23.1%
     50-54     22    7.521    9.425    4.181      22.7%
     55-59     26    5.926    7.738    0.982      38.5%
     60-64     41    3.986    5.310    0.977      36.6%
     65-69     37    3.539    4.796   -0.958      59.5%
     70-74     48    4.146    4.898   -0.345      45.8%
     75-79     36    3.416    4.404   -0.627      52.8%
     8

Testing: 100%|██████████| 288/288 [00:01<00:00, 198.73it/s]



📊 年齡組詳細表現分析 (fold4_mae_best.pth):
 Age Group  Count      MAE     RMSE     Bias  Low Est %
--------------------------------------------------------------
       5-9    152    1.308    2.046    0.554      36.8%
     10-14    133    1.725    2.103   -0.787      69.9%
     15-19    192    1.497    2.436    0.256      37.0%
     20-24    291    2.210    3.179   -1.238      84.2%
     25-29     79    4.283    5.232   -2.902      86.1%
     30-34     37    7.178    8.116   -4.046      83.8%
     35-39     14    7.661    8.895   -2.208      57.1%
     40-44     10    7.451    8.963   -0.681      40.0%
     45-49     13    6.043    7.789    1.625      46.2%
     50-54     22    6.679    9.037    3.813      18.2%
     55-59     26    6.104    8.341   -1.032      50.0%
     60-64     41    3.732    4.970   -1.082      61.0%
     65-69     37    3.760    4.606   -2.527      78.4%
     70-74     48    3.992    4.958   -2.978      72.9%
     75-79     36    4.582    5.798   -3.928      83.3%
     8

Testing: 100%|██████████| 288/288 [00:01<00:00, 195.71it/s]


📊 年齡組詳細表現分析 (fold5_mae_best.pth):
 Age Group  Count      MAE     RMSE     Bias  Low Est %
--------------------------------------------------------------
       5-9    152    1.282    1.669    0.716      32.2%
     10-14    133    1.565    1.991   -0.226      54.9%
     15-19    192    1.960    2.445    1.035      21.9%
     20-24    291    1.686    2.511   -0.158      64.3%
     25-29     79    4.212    6.242   -1.219      77.2%
     30-34     37    6.862    8.240   -2.306      70.3%
     35-39     14    7.056    8.130   -3.480      85.7%
     40-44     10    8.373    9.639   -5.730      80.0%
     45-49     13    9.720   12.214    2.017      30.8%
     50-54     22    7.322    8.835    3.228      36.4%
     55-59     26    6.401    7.935   -0.586      42.3%
     60-64     41    3.485    4.320    0.671      46.3%
     65-69     37    4.097    5.160   -1.912      70.3%
     70-74     48    3.796    4.659   -1.382      58.3%
     75-79     36    3.445    4.360   -2.231      75.0%
     8

In [ ]:
# 測試程式 - 5 個 fold 在測試集上推論
if __name__ == "__main__":
    loss_type = 'mae'
    
    print("="*60)
    print(f"開始測試 - 使用 {loss_type.upper()} 訓練的 5 個 fold 模型")
    print("="*60)
    
    # 收集所有 fold 的最佳模型
    best_models = glob.glob(f"{cfg.SAVE_DIR}/fold*_{loss_type}_best.pth")
    best_models = sorted(best_models)
    
    if len(best_models) == 0:
        print(f"錯誤：找不到模型檔案於 {cfg.SAVE_DIR}")
        print("請確認模型已訓練完成")
    else:
        print(f"\n找到 {len(best_models)} 個模型:")
        for model_path in best_models:
            print(f"  - {os.path.basename(model_path)}")
        
        print(f"\n開始在測試集上評估...")
        print(f"測試集: {cfg.TEST_CSV}")
        print("="*60)
        
        # 逐一分析每個 fold
        for i, model_path in enumerate(best_models, 1):
            print(f"\n[{i}/{len(best_models)}] 評估模型: {os.path.basename(model_path)}")
            print("-"*60)
            analyze_age_group_performance(model_path, cfg.TEST_CSV, cfg.IMG_DIR)
            print("-"*60)
        
        print("\n所有 fold 測試完成！")

In [ ]:
# 集成推論 - 平均 5 個 fold 模型
if __name__ == "__main__":
    loss_type = 'mae'
    
    print("="*60)
    print("開始集成推論 - 5 個 fold 模型平均")
    print("="*60)
    
    # 自動收集訓練好的模型
    weight_paths = sorted(glob.glob(f"{cfg.SAVE_DIR}/fold*_{loss_type}_best.pth"))
    
    if len(weight_paths) != cfg.N_SPLITS:
        print(f"警告：找到 {len(weight_paths)} 個模型，但需要 {cfg.N_SPLITS} 個")
        print("找到的模型:")
        for p in weight_paths:
            print(f"  - {os.path.basename(p)}")
        exit(1)
    
    print(f"\n使用的模型:")
    for i, p in enumerate(weight_paths, 1):
        print(f"  Fold {i}: {os.path.basename(p)}")
    
    # 載入測試資料
    test_df = pd.read_csv(cfg.TEST_CSV).reset_index(drop=True)
    test_ds = GlobalOnlyDataset(test_df, cfg.IMG_DIR)
    test_ld = DataLoader(test_ds, batch_size=cfg.BATCH_SIZE,
                         shuffle=False, num_workers=2, pin_memory=True)
    
    print(f"\n測試集大小: {len(test_ds)}")
    print("="*60)
    
    # 準備儲存每個 fold 的預測
    ensemble_preds = np.zeros((len(test_ds), cfg.N_SPLITS), dtype=np.float32)
    
    # 逐一載入模型並預測
    for k, w_path in enumerate(weight_paths):
        print(f"\n推論 Fold {k+1}: {os.path.basename(w_path)}")
        model = BrainAgeModelWithSex().to(device)
        model.load_state_dict(torch.load(w_path, map_location=device))
        model.eval()
        
        offset = 0
        with torch.no_grad():
            for imgs, _, sex, _ in tqdm(test_ld, desc=f"Fold{k+1}"):
                bs = imgs.size(0)
                preds = model(imgs.to(device), sex.to(device)).cpu().numpy()
                ensemble_preds[offset:offset+bs, k] = preds
                offset += bs
    
    # 計算平均預測
    final_pred = ensemble_preds.mean(axis=1)
    
    # 計算每個樣本的標準差（模型間的不確定性）
    pred_std = ensemble_preds.std(axis=1)
    
    # 儲存結果
    result_df = pd.DataFrame({
        "NIfTI_ID": test_df["NIfTI_ID"],
        "True_Age": test_df["AGE"],
        "Predicted_Age": final_pred,
        "Prediction_Std": pred_std
    })
    
    out_csv = f"{cfg.SAVE_DIR}/ensemble_predictions.csv"
    result_df.to_csv(out_csv, index=False)
    
    # 計算整體指標
    mae = np.mean(np.abs(final_pred - test_df["AGE"]))
    rmse = np.sqrt(np.mean((final_pred - test_df["AGE"])**2))
    
    print("\n" + "="*60)
    print("集成推論完成！")
    print("="*60)
    print(f"測試集 MAE:  {mae:.3f}")
    print(f"測試集 RMSE: {rmse:.3f}")
    print(f"平均預測標準差: {pred_std.mean():.3f}")
    print(f"\n結果已儲存: {out_csv}")